<a href="https://colab.research.google.com/github/15nisha/EXperimets/blob/main/tag_prediction_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to solve Multi - label classification Problem**

In [123]:
import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
import sklearn.metrics as skm


warnings.filterwarnings("ignore")

**Importing data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def impoting_dataset(path):
  ''' Import csv file  from google drive into colab'''
  data_raw = pd.read_csv(path)
  return data_raw


In [ ]:
 df = impoting_dataset('/content/drive/MyDrive/data.csv')

In [ ]:
df.head()

,Unnamed: 0,tags,description
0,0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,2,part-time-job,This is a great position for the right person....
3,3,licence-needed,A large multi-specialty health center is expan...
4,4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [ ]:
len(df)

4375

In [ ]:
df.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [ ]:
df.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [ ]:
data = df.copy()

In [ ]:
data.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


**Data Cleaning**

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

tags           0
description    0
dtype: int64

In [ ]:
# load stop words
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_word = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 **Cleaning on description part**

In [ ]:
def clean(text):
    from nltk.corpus import stopwords

    #     remove urls
    text = re.sub(r'http\S+', " ", text)

    #     remove mentions
    text = re.sub(r'@\w+',' ',text)

    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #     remove digits
    text = re.sub(r'\d+', ' ', text)

    #     remove html tags
    text = re.sub('r<.*?>',' ', text)
    
    #     remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])
    
      
    return text

In [ ]:
df['description'] = df['description'].apply(lambda x: clean(x))

In [ ]:
df.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer midstream service provide...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing accepting resumes Industrial Main...
2,part-time-job,This great position right person. HEALTHCAREse...
3,licence-needed,A large multi-specialty health center expandin...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director responsible ...


In [ ]:
token=ToktokTokenizer()
lemma=WordNetLemmatizer()

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

In [ ]:
df['description'] = df['description'].apply(
              lambda x: lemitizeWords(x)) 

**Cleaning on Tags parts**

In [ ]:
df['new_tags'] = df["tags"].apply(lambda x: x.split())

In [ ]:
all_tags = [item for sublist in df['new_tags'].values for item in sublist]

In [ ]:
len(all_tags)

6880

In [ ]:
# getting unique set
my_set = set(all_tags)
unique_tags = list(my_set)
len(unique_tags)

12

**visualisation**

**train _test split**

In [ ]:
X = df['description']
y = df['new_tags']

In [ ]:
x, X_test, y, y_test = train_test_split(X,y,test_size=0.2,train_size=0.8)
X_train, X_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [ ]:
len(X_test)

701

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    # test_size=0.2, random_state=0)

In [ ]:
mulitlabel_binarizer = MultiLabelBinarizer()
# y_bin = mulitlabel_binarizer.fit_transform(y)

In [ ]:
y_test = mulitlabel_binarizer.fit_transform(y_test)

In [ ]:
y_test

array([[0, 1, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 0, 1]])

In [ ]:
y_train = mulitlabel_binarizer.fit_transform(y_train)

In [ ]:
y_train

array([[0, 1, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 1]])

In [ ]:
y_cv = mulitlabel_binarizer.fit_transform(y_cv)

In [ ]:
y_cv

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vectorizer = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       max_features=1000)

In [ ]:
X_train = vectorizer.fit_transform(X_train)


In [ ]:
X_test = vectorizer.fit_transform(X_test)

In [ ]:
X_cv = vectorizer.fit_transform(X_cv)

**Training the model**

In [ ]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print( skm.classification_report(y_test,y_pred))
    print("---") 

In [ ]:
dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
mn = MultinomialNB()
svc = LinearSVC()
perceptron = Perceptron()
pac = PassiveAggressiveClassifier()
rfc = RandomForestClassifier()

for classifier in [dummy, sgd, lr, mn, svc, perceptron, pac,rfc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score(y_pred, classifier)

Clf:  DummyClassifier
Jacard score: 11.637456694518034
Hamming loss: 26.29576795054684
              precision    recall  f1-score   support

           0       0.13      0.14      0.13        74
           1       0.28      0.27      0.28       209
           2       0.13      0.16      0.14       111
           3       0.04      0.05      0.04        44
           4       0.26      0.28      0.27       180
           5       0.22      0.23      0.22       173
           6       0.11      0.10      0.11        88
           7       0.14      0.14      0.14       109
           8       0.05      0.06      0.05        18
           9       0.13      0.10      0.11        79
          10       0.20      0.21      0.21       123
          11       0.25      0.23      0.24       154

   micro avg       0.20      0.20      0.20      1362
   macro avg       0.16      0.16      0.16      1362
weighted avg       0.20      0.20      0.20      1362
 samples avg       0.18      0.19      0.17    

**Hyperparameter tuning**

In [143]:
param_grid = {'estimator__C':[1,10,100,1000]}

svc = OneVsRestClassifier(LinearSVC())
CV_svc = model_selection.GridSearchCV(estimator=svc, 
                                      param_grid=param_grid, cv= 5, 
                                      verbose=10)
CV_svc.fit(X_cv, y_cv)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] estimator__C=1 ..................................................
[CV] ...................... estimator__C=1, score=0.113, total=   0.1s
[CV] estimator__C=1 ..................................................
[CV] ...................... estimator__C=1, score=0.114, total=   0.1s
[CV] estimator__C=1 ..................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ...................... estimator__C=1, score=0.093, total=   0.1s
[CV] estimator__C=1 ..................................................
[CV] ...................... estimator__C=1, score=0.064, total=   0.1s
[CV] estimator__C=1 ..................................................
[CV] ...................... estimator__C=1, score=0.150, total=   0.1s
[CV] estimator__C=10 .................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.4s remaining:    0.0s


[CV] ..................... estimator__C=10, score=0.099, total=   0.1s
[CV] estimator__C=10 .................................................
[CV] ..................... estimator__C=10, score=0.100, total=   0.1s
[CV] estimator__C=10 .................................................


...

[CV] ..................... estimator__C=10, score=0.129, total=   0.1s
[CV] estimator__C=10 .................................................
[CV] ..................... estimator__C=10, score=0.086, total=   0.1s
[CV] estimator__C=10 .................................................


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.9s remaining:    0.0s


[CV] ..................... estimator__C=10, score=0.143, total=   0.1s
[CV] estimator__C=100 ................................................
[CV] .................... estimator__C=100, score=0.106, total=   0.2s
[CV] estimator__C=100 ................................................
[CV] .................... estimator__C=100, score=0.093, total=   0.2s
[CV] estimator__C=100 ................................................
[CV] .................... estimator__C=100, score=0.121, total=   0.2s
[CV] estimator__C=100 ................................................
[CV] .................... estimator__C=100, score=0.071, total=   0.1s
[CV] estimator__C=100 ................................................
[CV] .................... estimator__C=100, score=0.129, total=   0.2s
[CV] estimator__C=1000 ...............................................
[CV] ................... estimator__C=1000, score=0.106, total=   0.2s
[CV] estimator__C=1000 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.7s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                               class_weight=None,
                                                               dual=True,
                                                               fit_intercept=True,
                                                               intercept_scaling=1,
                                                               loss='squared_hinge',
                                                               max_iter=1000,
                                                               multi_class='ovr',
                                                               penalty='l2',
                                                               random_state=None,
                                                               tol=0.0001,
                                                               verbose=0),
                 

In [129]:
best_model = CV_svc.best_estimator_

best_model

OneVsRestClassifier(estimator=LinearSVC(C=10, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='squared_hinge', max_iter=1000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0),
                    n_jobs=None)

In [134]:
y_pred = best_model.predict(X_test)

print_score(y_pred, best_model)
# hamming loss is fraction which is wrongly predicted

Clf:  OneVsRestClassifier
Jacard score: 10.511174512601047
Hamming loss: 19.65049928673324
              precision    recall  f1-score   support

           0       0.14      0.04      0.06        74
           1       0.20      0.08      0.12       209
           2       0.25      0.25      0.25       111
           3       0.14      0.02      0.04        44
           4       0.42      0.23      0.30       180
           5       0.32      0.16      0.21       173
           6       0.20      0.09      0.13        88
           7       0.39      0.08      0.14       109
           8       0.00      0.00      0.00        18
           9       0.27      0.04      0.07        79
          10       0.18      0.10      0.13       123
          11       0.29      0.19      0.23       154

   micro avg       0.28      0.13      0.18      1362
   macro avg       0.23      0.11      0.14      1362
weighted avg       0.27      0.13      0.17      1362
 samples avg       0.18      0.13      0.14

In [133]:
for i in range(y_train.shape[1]):
    print(mulitlabel_binarizer.classes_[i])
    print(confusion_matrix(y_test[:,i], y_pred[:,i]))
    print("")

1-year-experience-needed
[[609  18]
 [ 71   3]]

2-4-years-experience-needed
[[426  66]
 [192  17]]

5-plus-years-experience-needed
[[506  84]
 [ 83  28]]

associate-needed
[[651   6]
 [ 43   1]]

bs-degree-needed
[[463  58]
 [138  42]]

full-time-job
[[471  57]
 [146  27]]

hourly-wage
[[581  32]
 [ 80   8]]

licence-needed
[[578  14]
 [100   9]]

ms-or-phd-needed
[[683   0]
 [ 18   0]]

part-time-job
[[614   8]
 [ 76   3]]

salary
[[522  56]
 [111  12]]

supervising-job
[[476  71]
 [125  29]]



**Feature importance**

In [135]:
def print_top10(feature_names, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("--------------------------------------------")
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))
        print("--------------------------------------------")

In [136]:
feature_names = vectorizer.get_feature_names() 

In [137]:
print_top10(feature_names, best_model,
            mulitlabel_binarizer.classes_)

--------------------------------------------
1-year-experience-needed: principles coach drive diploma service world patient accredit one year
--------------------------------------------
--------------------------------------------
2-4-years-experience-needed: server efficiency skills exceed equipment accounts restaurant prefer two years
--------------------------------------------
--------------------------------------------
5-plus-years-experience-needed: broker repair letter leadership highly key five complex responsible years
--------------------------------------------
--------------------------------------------
associate-needed: members associate assistant visit relate education student payroll degree associates
--------------------------------------------
--------------------------------------------
bs-degree-needed: relationship approach assign communications regulations bs bachelors science degree bachelor
--------------------------------------------
-------------------------

**Testing**

In [138]:
classes = mulitlabel_binarizer.classes_

In [139]:
classes[7]

'licence-needed'

In [140]:
# to match index with class name
def print_class(arr):
  for index, value in enumerate(arr):
    if value == 1:
      print(classes[index])   
    

In [141]:
test = """
this job is for driver you'll have to have a bike in order to deliver the products
and also you need a helmat for your safety and need a license and it's also part time
"""

# test = df['description'][549]

# clean
test  = clean(test)

# tfidf
tf = vectorizer.transform([test])

# predict
pred = best_model.predict(tf)[0]
pred = list(pred)

print_class(pred)

# predict the output
# print_class(best_model.predict(tf))

licence-needed
part-time-job
